## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_1_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_exec_csv/"+'gcc6'+"_1_32_exec.csv", index_col=0)

# 형태 출력
print(gcc6_1_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_1_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_1_32.head()

(10532416, 2)
reset_index 완료
input data shape


,bin,label
0,53,1
1,83,0
2,ec,0
3,08,0
4,e8,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_1_32['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_1_32['label'].value_counts())

256
0    10494535
1       37881
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
######################## 
idx = gcc6_1_32[gcc6_1_32['label']==1].index  # 407, 474 ...
ls = list(idx)

# 최종 뽑을 행에 대한 index
ls_idx = [] 
left_idx, right_idx = 0, 32 # 3개씩

# 6gram
for k in range(left_idx, right_idx):
    ls_idx.extend(list(idx + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx = list(filter(lambda x: x<len(gcc6_1_32), ls_idx))
print(len(ls_idx))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub = len(ls_idx)%(right_idx)
print('나머지', sub)

ls_idx = ls_idx[:len(ls_idx)-sub]
print('최종 길이', len(ls_idx))

print('gcc6_1_32', len(ls_idx))

# loc 로 수정필요
gcc6_1_32_Ngram = gcc6_1_32.loc[ls_idx,:].copy()

1212180
나머지 20
최종 길이 1212160
gcc6_0_32 1212160


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성
# random index
random_idx = np.random.randint(len(gcc6_1_32)-right_idx)

# 목표치
goal = len(gcc6_1_32_Ngram)/right_idx
count=0

d = pd.DataFrame(columns=gcc6_1_32.columns)

# goal 에 도달할 때까지
while count!=goal:
    
    # 진행상황 살펴보기 위함
    if count%1000==0:
        print(count, end=' ')
        
    # 랜덤 N 바이트씩 뽑음
    df = gcc6_1_32[random_idx:random_idx+right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df['label']:
        d = pd.concat([d, df])
        count+=1

print('완료')

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 완료


## (5) False Data + True Data 합치기

In [5]:
ff = pd.DataFrame(columns=gcc6_1_32.columns)

for i in range(0, int(len(d)/right_idx)):
    ff = pd.concat([ff, gcc6_1_32[i*right_idx:(i+1)*right_idx], d[i*right_idx:(i+1)*right_idx]])
    
    if i%1000==0:
        print(i, end=' ')
ff.shape

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 

(2424320, 2)

## (6) one hot encoding

In [6]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_1_32_onehot_Ngram = pd.get_dummies(ff['bin'])
gcc6_1_32_onehot_Ngram = pd.concat([ff['label'], gcc6_1_32_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(gcc6_1_32_onehot_Ngram.shape)

원핫인코딩완료
(2424320, 257)


In [7]:
# 훈련 데이터, 훈련 라벨
x_gcc6_1_32_3 = gcc6_1_32_onehot_Ngram.iloc[:,1:].to_numpy()
y_gcc6_1_32_3 = gcc6_1_32_onehot_Ngram['label'].to_numpy()
print(x_gcc6_1_32_3.shape, y_gcc6_1_32_3.shape)

x_gcc6_1_32_3 = x_gcc6_1_32_3.reshape(-1, right_idx, x_gcc6_1_32_3.shape[1])
y_gcc6_1_32_3 = y_gcc6_1_32_3.reshape(-1, right_idx, 1)

print(x_gcc6_1_32_3.shape, y_gcc6_1_32_3.shape)

(2424320, 256) (2424320,)
(75760, 32, 256) (75760, 32, 1)


## (7) 모델

In [8]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

Using TensorFlow backend.


## (8) 학습 - 10 KFold

In [9]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_1_32_3, y_gcc6_1_32_3):
    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_gcc6_1_32_3[train],
               y_gcc6_1_32_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_1_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_1_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
68184/68184 [==============================] - 53s 784us/step - loss: 0.0083 - accuracy: 0.9982
Epoch 2/10
68184/68184 [==============================] - 53s 778us/step - loss: 0.0041 - accuracy: 0.9988
Epoch 3/10
68184/68184 [=======================

accuracy_score 0.9991379025871172
recall_score 0.6532258064516129
precision_score 0.7523219814241486
f1_score 0.6992805755395683
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 51s 744us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 2/10
68184/68184 [==============================] - 50s 739us/step - loss: 0.0020 - accuracy: 0.9994
Epoch 3/10
68

accuracy_score 0.9993111470432946
recall_score 0.6839237057220708
precision_score 0.8311258278145696
f1_score 0.750373692077728
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 51s 745us/step - loss: 9.2408e-04 - accuracy: 0.9997
Epoch 2/10
68184/68184 [==============================] - 50s 736us/step - loss: 7.7821e-04 - accuracy: 0.9998
Epoch 

accuracy_score 0.9995462645195353
recall_score 0.854066985645933
precision_score 0.8793103448275862
f1_score 0.866504854368932
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 52s 763us/step - loss: 4.8421e-04 - accuracy: 0.9999
Epoch 2/10
68184/68184 [==============================] - 53s 780us/step - loss: 2.9715e-04 - accuracy: 0.9999s - loss

accuracy_score 0.9997318835797254
recall_score 0.8790322580645161
precision_score 0.9423631123919308
f1_score 0.9095966620305981
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 55s 811us/step - loss: 2.3344e-04 - accuracy: 0.9999
Epoch 2/10
68184/68184 [==============================] - 55s 800us/step - loss: 1.3312e-04 - accuracy: 1.0000
Epoch

...

accuracy_score 0.9999587513199577
recall_score 0.9775280898876404
precision_score 0.9942857142857143
f1_score 0.9858356940509916
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 59s 872us/step - loss: 6.6193e-05 - accuracy: 1.0000
Epoch 2/10
68184/68184 [==============================] - 59s 862us/step - loss: 3.8002e-05 - accuracy: 1.0000
Epoch

accuracy_score 0.9999587513199577
recall_score 0.9811827956989247
precision_score 0.9918478260869565
f1_score 0.9864864864864865
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 61s 894us/step - loss: 5.7471e-05 - accuracy: 1.0000
Epoch 2/10
68184/68184 [==============================] - 60s 882us/step - loss: 4.6565e-05 - accuracy: 1.0000
Epoch

accuracy_score 0.999962876187962
recall_score 0.9828850855745721
precision_score 0.995049504950495
f1_score 0.9889298892988929
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 62s 906us/step - loss: 3.1149e-05 - accuracy: 1.0000
Epoch 2/10
68184/68184 [==============================] - 61s 888us/step - loss: 2.9300e-05 - accuracy: 1.0000
Epoch 3

accuracy_score 0.9999958751319957
recall_score 0.9971264367816092
precision_score 1.0
f1_score 0.9985611510791367
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
68184/68184 [==============================] - 67s 977us/step - loss: 1.9148e-05 - accuracy: 1.0000
Epoch 2/10
68184/68184 [==============================] - 65s 957us/step - loss: 1.1493e-05 - accuracy: 1.0000
Epoch 3/10
68184/68

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9991379025871172, 0.9993111470432946, 0.9995462645195353, 0.9997318835797254, 0.9998845036958818, 0.9999587513199577, 0.9999587513199577, 0.999962876187962, 0.9999958751319957, 1.0]

K-fold cross validation Recall: [0.6532258064516129, 0.6839237057220708, 0.854066985645933, 0.8790322580645161, 0.952755905511811, 0.9775280898876404, 0.9811827956989247, 0.9828850855745721, 0.9971264367816092, 1.0]

K-fold cross validation Precision: [0.7523219814241486, 0.8311258278145696, 0.8793103448275862, 0.9423631123919308, 0.9731903485254692, 0.9942857142857143, 0.9918478260869565, 0.995049504950495, 1.0, 1.0]

K-fold cross validation F1-Score: [0.6992805755395683, 0.750373692077728, 0.866504854368932, 0.9095966620305981, 0.9628647214854111, 0.9858356940509916, 0.9864864864864865, 0.9889298892988929, 0.9985611510791367, 1.0]


## (9) 평가지표

In [10]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9997487955385427
10-Fold Cross_validation. Recall : 0.896172706933869
10-Fold Cross_validation. Precision : 0.9359494660306871
10-Fold Cross_validation. F1-Score : 0.9148433726417744
